In [1]:
from git import Repo

In [2]:
def get_repo(path="."):
    repo = Repo(path)
    return repo

In [3]:
repo = get_repo("..")

In [4]:
def get_untracked_files(repo):
    for f in repo.untracked_files:
        yield f

In [5]:
untracked_files = get_untracked_files(repo)

In [6]:
list(untracked_files)

['Untitled.ipynb', 'existing_db.db', 'lab/vcs.ipynb']

In [7]:
def changed_staged_files(repo):
    """
    Yields paths of staged files which have been added, modified or renamed.
    Files which have been modified and renamed are yielded only once.
    """
    hcommit = repo.head.commit
    changes = hcommit.diff()
    # TODO: improve performance and get rid of mutable set
    files = set()
    for f in changes.iter_change_type('A'):
        files.add(f.b_path)
    for f in changes.iter_change_type('M'):
        files.add(f.b_path)
    for f in changes.iter_change_type('R'):
        files.add(f.b_path)
    for f in files:
        yield f
    del files  # free memory from set

In [8]:
staged_files = changed_staged_files(repo)
list(staged_files)

['Pipfile', 'Pipfile.lock_new']

In [9]:
def changed_unstaged_files(repo):
    """
    Yields paths of unstaged files (versioned files in working tree)
    which have been modified or renamed.
    Files which have been modified and renamed are yielded only once.
    """
    changes = repo.index.diff(None)
    # TODO: improve performance and get rid of mutable set
    files = set()
    for f in changes.iter_change_type('M'):
        files.add(f.b_path)
    for f in changes.iter_change_type('R'):
        files.add(f.b_path)
    for f in files:
        yield f
    del files  # free memory from set

In [10]:
unstaged_files = changed_unstaged_files(repo)
list(unstaged_files)

['lab/impact.ipynb']

In [11]:
def changed_files_non_ci(repo):
    """
    File changes considered for tia invocation not in CI are unstaged and
    staged file changes.
    """
    unstaged_files = changed_unstaged_files(repo)
    staged_files = changed_staged_files(repo)
    yield from unstaged_files
    yield from staged_files

In [12]:
non_ci_files = changed_files_non_ci(repo)
list(non_ci_files)

['lab/impact.ipynb', 'Pipfile', 'Pipfile.lock_new']

In [13]:
def changed_files_ci(repo):
    """
    File changes considered for tia invocation in CI are changed files between
    the current HEAD and the last commit.
    Yields paths of new, modified and renamed files.
    Files which have been modified and renamed are yielded only once.
    """
    hcommit = repo.head.commit
    changes = hcommit.diff('HEAD~1')
    # TODO: improve performance and get rid of mutable set
    files = set()
    for f in changes.iter_change_type('A'):
        files.add(f.b_path)
    for f in changes.iter_change_type('M'):
        files.add(f.b_path)
    for f in changes.iter_change_type('R'):
        files.add(f.b_path)
    for f in files:
        yield f
    del files  # free memory from set

In [14]:
ci_files = changed_files_ci(repo)
list(ci_files)

['tests/test_config.py',
 'tia/cov.py',
 'tests/test_cov.py',
 'tests/test_cli.py',
 'tia/cli.py',
 'tests/test_env.py',
 'tia/config.py',
 'setup.py']